Multiple linear regression

In [ ]:
pip install pyspark

In [ ]:
import pyspark
from pyspark.sql import SparkSession
#SparkSession is now the entry point of Spark
#SparkSession can also be construed as gateway to spark libraries

#create instance of spark class
spark=SparkSession.builder.appName('housing_price_model').getOrCreate()

#create spark dataframe of input csv file
df=spark.read.csv('/content/cruise_ship_info.csv',inferSchema=True,header=True)
df.show(10)

+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
|  Ship_name|Cruise_line|Age|           Tonnage|passengers|length|cabins|passenger_density|crew|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
|    Journey|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|
|      Quest|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|
|Celebration|   Carnival| 26|            47.262|     14.86|  7.22|  7.43|             31.8| 6.7|
|   Conquest|   Carnival| 11|             110.0|     29.74|  9.53| 14.88|            36.99|19.1|
|    Destiny|   Carnival| 17|           101.353|     26.42|  8.92| 13.21|            38.36|10.0|
|    Ecstasy|   Carnival| 22|            70.367|     20.52|  8.55|  10.2|            34.29| 9.2|
|    Elation|   Carnival| 15|            70.367|     20.52|  8.55|  10.2|            34.29| 9.2|
|    Fantasy|   Carnival| 23| 

In [ ]:
#prints structure of dataframe along with datatype
df.printSchema()

root
 |-- Ship_name: string (nullable = true)
 |-- Cruise_line: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Tonnage: double (nullable = true)
 |-- passengers: double (nullable = true)
 |-- length: double (nullable = true)
 |-- cabins: double (nullable = true)
 |-- passenger_density: double (nullable = true)
 |-- crew: double (nullable = true)



In [ ]:
#In our predictive model, below are the columns
df.columns

['Ship_name',
 'Cruise_line',
 'Age',
 'Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density',
 'crew']

In [ ]:
#columns identified as features are as below:
#['Cruise_line','Age','Tonnage','passengers','length','cabins','passenger_density']
#to work on the features, spark MLlib expects every value to be in numeric form
#feature 'Cruise_line is string datatype
#using StringIndexer, string type will be typecast to numeric datatype
#import library strinindexer for typecasting

from pyspark.ml.feature import StringIndexer
indexer=StringIndexer(inputCol='Cruise_line',outputCol='cruise_cat')
indexed=indexer.fit(df).transform(df)


In [ ]:
#above code will convert string to numeric feature and create a new dataframe
#new dataframe contains a new feature 'cruise_cat' and can be used further
#feature cruise_cat is now vectorized and can be used to fed to model
for item in indexed.head(5):
    print(item)
    print('\n')

Row(Ship_name='Journey', Cruise_line='Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55, cruise_cat=16.0)


Row(Ship_name='Quest', Cruise_line='Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55, cruise_cat=16.0)


Row(Ship_name='Celebration', Cruise_line='Carnival', Age=26, Tonnage=47.262, passengers=14.86, length=7.22, cabins=7.43, passenger_density=31.8, crew=6.7, cruise_cat=1.0)


Row(Ship_name='Conquest', Cruise_line='Carnival', Age=11, Tonnage=110.0, passengers=29.74, length=9.53, cabins=14.88, passenger_density=36.99, crew=19.1, cruise_cat=1.0)


Row(Ship_name='Destiny', Cruise_line='Carnival', Age=17, Tonnage=101.353, passengers=26.42, length=8.92, cabins=13.21, passenger_density=38.36, crew=10.0, cruise_cat=1.0)




In [ ]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
#creating vectors from features
#Apache MLlib takes input if vector form
assembler=VectorAssembler(inputCols=['Age','Tonnage','passengers','length','cabins','passenger_density','cruise_cat'],outputCol='features')
output=assembler.transform(indexed)
output.select('features','crew').show(5)

+--------------------+----+
|            features|crew|
+--------------------+----+
|[6.0,30.276999999...|3.55|
|[6.0,30.276999999...|3.55|
|[26.0,47.262,14.8...| 6.7|
|[11.0,110.0,29.74...|19.1|
|[17.0,101.353,26....|10.0|
+--------------------+----+
only showing top 5 rows



In [ ]:

#final data consist of features and label which is crew.
final_data=output.select('features','crew')
#splitting data into train and test
train_data,test_data=final_data.randomSplit([0.7,0.3])
train_data.describe().show()

+-------+------------------+
|summary|              crew|
+-------+------------------+
|  count|               105|
|   mean| 7.742666666666679|
| stddev|3.6877323307948666|
|    min|              0.59|
|    max|              21.0|
+-------+------------------+



In [ ]:
test_data.describe().show()

+-------+------------------+
|summary|              crew|
+-------+------------------+
|  count|                53|
|   mean| 7.896226415094341|
| stddev|3.1376088558591473|
|    min|              0.59|
|    max|              13.6|
+-------+------------------+



In [ ]:
#import LinearRegression library
from pyspark.ml.regression import LinearRegression

#creating an object of class LinearRegression
#object takes features and label as input arguments
ship_lr=LinearRegression(featuresCol='features',labelCol='crew')

#pass train_data to train model
trained_ship_model=ship_lr.fit(train_data)

#evaluating model trained for Rsquared error
ship_results=trained_ship_model.evaluate(train_data)


In [ ]:
print('Rsquared Error :',ship_results.r2)

Rsquared Error : 0.9350721892853087


In [ ]:
trained_ship_model_test=ship_lr.fit(test_data)

#evaluating model trained for Rsquared error
ship_results_test=trained_ship_model_test.evaluate(train_data)

print('Rsquared Error :',ship_results_test.r2)

Rsquared Error : 0.9296141746626077


In [ ]:
#testing Model on unlabeled data
#create unlabeled data from test_data
#testing model on unlabeled data
unlabeled_data=test_data.select('features')
unlabeled_data.show(5)


+--------------------+
|            features|
+--------------------+
|[6.0,30.276999999...|
|[6.0,113.0,37.82,...|
|[7.0,89.6,25.5,9....|
|[7.0,158.0,43.7,1...|
|[8.0,77.499,19.5,...|
+--------------------+
only showing top 5 rows



In [ ]:
predictions=trained_ship_model.transform(unlabeled_data)
predictions.show()
#below are the results of output from test data

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|[6.0,30.276999999...|  4.22362910754293|
|[6.0,113.0,37.82,...|11.599891893511957|
|[7.0,89.6,25.5,9....|11.002723968383277|
|[7.0,158.0,43.7,1...|13.862651293517747|
|[8.0,77.499,19.5,...| 8.525180474635443|
|[8.0,91.0,22.44,9...|10.013959672028223|
|[9.0,59.058,17.0,...| 7.523393989636261|
|[9.0,81.0,21.44,9...| 9.452691698930208|
|[9.0,88.5,21.24,9...|  9.48187432653474|
|[9.0,105.0,27.2,8...|11.262987691763682|
|[10.0,46.0,7.0,6....|2.7522263542467633|
|[10.0,58.825,15.6...| 7.206618747241352|
|[10.0,77.0,20.16,...| 8.723944013563656|
|[10.0,81.76899999...| 8.697602722550876|
|[10.0,105.0,27.2,...|11.259756980951286|
|[10.0,110.0,29.74...|12.089222240952523|
|[11.0,85.0,18.48,...| 8.699080474212181|
|[11.0,90.0,22.4,9...| 9.981524281382475|
|[11.0,90.09,25.01...| 8.806871302899925|
|[11.0,91.62700000...| 9.132304321187917|
+--------------------+------------